In [1]:
%config IPCompleter.greedy=True

In [2]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt
import pylab as pl

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re

from sklearn.cross_validation import cross_val_score
from sklearn.metrics import average_precision_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.pipeline import Pipeline

from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
from sklearn import datasets
iris = datasets.load_iris()



/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [3]:
train = pd.read_csv('./Data/emails.train.csv')
text = train['text']

Models = []
Scores = []
Cross_scores = []



# Cleaning Data

In [4]:
re_tokenizer = RegexpTokenizer(r'\b[^\d\W]+\b') # tokenize words that are not numbers
mystopwords = set(stopwords.words('english')) # stop words remover
extras = set(['_', 'subject']) # remove subject and _
mystopwords.update(extras)
#ps = PorterStemmer() # stemming words
#stemmer = SnowballStemmer("english") # stemming words
lemmatizer = WordNetLemmatizer() # lemmatizing words

In [5]:
## First, lemmatize and create list of words
def lemma(df):
    # lemmatize 
    for i, line in enumerate(df['text']): 
        newline = [] 
        words = line.split() 
        for word in words: 
            word = lemmatizer.lemmatize(word) 
            newline.append(word) 
        new = ' '.join(newline)  
        df.loc[i, 'text'] = new # replace line at the index position with the new line that is the lemmatized words

In [6]:
## data
train = pd.read_csv('./Data/emails.train.csv')
test  = pd.read_csv('./Data/emails.test.csv')

lemma(train)
lemma(test)

# for i in 10:

# Get labels splitted set
subtrain_X, subval_X = train_test_split(train, test_size = 0.25, )

Y_train = train['spam']
Y_test = test['spam']

## tf-idf vectorizer

In [7]:
vectorizer = TfidfVectorizer(stop_words=mystopwords, tokenizer=re_tokenizer.tokenize, max_features = 1000)
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text']).todense() ## features
X_test = vectorizer.transform(test['text']).todense() ## features

# KNN model classifier

In [72]:
random_k = list(range(1,50))

# only odd numbers
neighbor = filter(lambda x: x % 2 != 0, random_k)

cross_scores = []

# 10 fold crossvalidation using training set, to find optimal k-value for classifier
for k in neighbor:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
    cross_scores.append(scores.mean())

# misclassification error
MSE = [1 - x for x in cross_scores]
optimal_k = neighbor[MSE.index(min(MSE))]
print "The optimal number of neighbors is %d" % optimal_k

KeyboardInterrupt: 

In [ ]:
# plot with MSE vs k
plt.plot(neighbor, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [73]:
# with use of optimal_k
knn = KNeighborsClassifier(n_neighbors= optimal_k)

# model fitting
knn.fit(X_train, Y_train)

# predict
Y_pred = knn.predict(X_test)

# accuracy evaluation
print accuracy_score(Y_test, Y_pred)
Models.append('KNN')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(cross_val_score(knn, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('knn.csv', index=False)

NameError: name 'optimal_k' is not defined

In [ ]:
h = .02  # step size in the mesh
 
# Calculate min, max and limits
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
 
# Put the result into a color plot
plt.figure()
plt.scatter(X_train[:, 0], X_train[:, 1])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("Data points")
plt.show()

# Random Forest Model Classifier

In [8]:
rf_class = RandomForestClassifier() 
 
# Use a grid search to find optimal parameters
param_grid = { 
           "n_estimators" : [8, 16, 32, 40, 48, 56],
           "max_depth" : [1, 5, 10, 15, 20, 25],
           "min_samples_leaf" : [1, 2, 4, 6, 8, 10]}
 
CV_rf_class = GridSearchCV(estimator=rf_class, param_grid=param_grid, n_jobs=-1)
CV_rf_class.fit(X_train, Y_train)
print CV_rf_class.best_params_


{'n_estimators': 48, 'max_depth': 25, 'min_samples_leaf': 1}


In [11]:
print CV_rf_class.best_score_
# print accuracy_score(Y_test, Y_pred)


0.973141009699


In [12]:
est = CV_rf_class.best_estimator_
Y_pred = est.predict(X_test)
Models.append('Random Forest')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(cross_val_score(rf_class, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('rforest.csv', index=False)

In [14]:
print Scores

[0.97070884592852957]


# Gaussian Process Model Classifier

In [15]:
gaus_class = GaussianProcessClassifier()

gaus_class.fit(X_train, Y_train)

Y_pred = gaus_class.predict(X_test)

print accuracy_score(Y_test, Y_pred)

Models.append('Gaussian Process')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(cross_val_score(gaus_class, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('gaussian.csv', index=False)


0.960749853544


# Naive Bayes classifier for multivariate Bernoulli models

In [16]:
from sklearn.naive_bayes import BernoulliNB
NB = BernoulliNB()

NB.fit(X_train, Y_train)

Y_pred = NB.predict(X_test)

print (accuracy_score(Y_test, Y_pred))

Models.append('Bernoulli Naive Bayes')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(cross_val_score(NB, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('NB.csv', index=False)

0.932044522554


# Isolation Forest Model 

In [21]:
from sklearn.ensemble import IsolationForest

isolationForest = IsolationForest()

isolationForest.fit(X_train, Y_train)

y_pred = isolationForest.predict(X_test)

print (accuracy_score(Y_test, Y_pred))

Models.append('Isolation Forest')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(accuracy_score(Y_test, Y_pred))

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('isolationForest.csv', index=False)

0.984182776801


# C Support Vector Classification

In [18]:
#Import Library
from sklearn import svm

model = svm.SVC(kernel='rbf', C=1.0, gamma=1) 

model.fit(X_train, Y_train)
model.score(X_train, Y_train)
Y_pred = model.predict(X_test)

print (cross_val_score(model, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())
# print (accuracy_score(Y_test, Y_pred))

Models.append('C-Support Vector')
Scores.append(accuracy_score(Y_test, Y_pred))
Cross_scores.append(cross_val_score(model, X_train, Y_train, cv = 10, scoring = 'accuracy').mean())

pd.DataFrame({
    'id': test.id,
    'spam': Y_pred
}).to_csv('C-Support.csv', index=False)


0.986575170102


# Plot scores

In [30]:
plotly.tools.set_credentials_file(username='fvs10533176', api_key='r56zsrqviN0WhBa3R05F')

trace1 = go.Bar(
    x=Models,
    y=Scores,
    name='Accuracy'
)

trace2 = go.Bar(
    x=Models,
    y= Cross_scores,
    name='Accuracy with cross validation'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title= 'Accuracy scores for Spam detection',
    yaxis=dict(
        type='log',
        autorange=True
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Scores')



0.211718084515
